## Imports

In [ ]:
from tqdm import tqdm_notebook
import pickle
import os
import numpy as np
import pandas as pd

In [ ]:
import plotly.plotly as py
import plotly.tools as tools
import plotly.figure_factory as ff
import plotly.graph_objs as go

## Config

In [ ]:
tools.set_credentials_file(username='tdehaene', api_key='IM7S3UeptJgLHAUbLskn')

In [ ]:
mapping_classes = {0:'daisy', 1:'dandelion', 2:'rose', 3:'sunflower', 4:'tulip'}

### Plot dataset sizes

In [ ]:
train_size = sorted([(1, 854), (4, 797), (2, 624), (0, 597), (3, 584)], key=lambda x: x[0])
valid_size = sorted([(1, 198), (4, 186), (0, 171), (2, 160), (3, 150)], key=lambda x: x[0])

In [ ]:
trace1 = go.Bar(
    x=list(mapping_classes.values()),
    y=[x[1] for x in train_size],
    name='Training set'
)
trace2 = go.Bar(
    x=list(mapping_classes.values()),
    y=[x[1] for x in valid_size],
    name='Validation set'
)

data = [trace1, trace2]
layout = go.Layout(
    barmode='stack'
)

fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename='Training and validation size')

## Results visualisation

### Learning rate experiment 1

In [ ]:
with open("../logs/learning_rates/flowers_lr_find_loss.pickle","rb") as file:
    lr_find_loss = pickle.load(file)
    
with open("../logs/learning_rates/flowers_lr_find_lr.pickle","rb") as file:
    lr_find_lr = pickle.load(file)
 
traces = []

layout = go.Layout(
    xaxis=dict(
        showgrid=True,
        showline=False,
        type='log',
        showticklabels=True,
        tickcolor='rgb(127,127,127)',
        ticks='outside',
        zeroline=False,
        title = "learning rate [log]"
    ),
    yaxis=dict(
        showgrid=True,
        showline=False,
        showticklabels=True,
        tickcolor='rgb(127,127,127)',
        ticks='outside',
        zeroline=False,
        title = "loss"
    ),
)

traces.append(go.Scatter(
        x=lr_find_lr,
        y=lr_find_loss,
        showlegend=False,
        name="loss",
    ))

fig = go.Figure(data=traces, layout=layout)
py.iplot(fig, filename='td_medium_lr_loss_vanilla')

In [ ]:
with open("../logs/learning_rates/flowers_lr_find_lr.pickle","rb") as file:
    lr_find_lr = pickle.load(file)
 
traces = []

layout = go.Layout(
    xaxis=dict(
        showgrid=True,
        showline=False,
        showticklabels=True,
        tickcolor='rgb(127,127,127)',
        ticks='outside',
        zeroline=False,
        title = "step"
    ),
    yaxis=dict(
        showgrid=True,
        showline=False,
        showticklabels=True,
        tickcolor='rgb(127,127,127)',
        ticks='outside',
        zeroline=False,
        title = "learning rate"
    ),
)

traces.append(go.Scatter(
        x=list(range(len(lr_find_lr))),
        y=lr_find_lr,
        showlegend=False,
        name="loss",
    ))

fig = go.Figure(data=traces, layout=layout)
py.iplot(fig, filename='td_medium_lr_step_vanilla')

### Accuracy experiment 1

In [ ]:
loss_plots = {}
for i, file in enumerate(os.listdir("../logs/flowers_vanilla/")):
    opt = file.split("_")[1]
    
    plots = pickle.load(open(os.path.join("../logs","flowers_vanilla",file),"rb"))
    
    if opt == "adam" and not "adam" in loss_plots:
        plots = [{
            "acc":x['acc']/100.0,
            "loss":x["loss"],
            "val_loss":x["val_loss"],
            "val_acc":x["val_acc"]
        } for x in plots]
        
        plots = plots[:150]

    
    if opt in loss_plots:
        loss_plots[opt].append(plots)
    else:
        loss_plots[opt] = [plots]


#### Training accuracy

In [ ]:
traces = []
metric = "acc"
for key,val in loss_plots.items():
    
    x = list(np.arange(1,len(val[0])+1))
    x_rev = x[::-1]
        
    metric_values = np.asarray([[epoch[metric] for epoch in run] for run in val])
    
    y_upper = list(np.amax(metric_values,axis=0))
    y_lower = np.amin(metric_values,axis=0)
    y_lower = list(y_lower[::-1])
    
    traces.append(go.Scatter(
        x=x+x_rev,
        y=y_upper+y_lower,
        fill='tozerox',
        #fillcolor='rgba(0,100,80,0.2)',
        #line=dict(color='rgba(255,255,255,0)'),
        showlegend=True,
        name=key,
    ))

In [ ]:
layout = go.Layout(
    paper_bgcolor='rgb(255,255,255)',
    plot_bgcolor='rgb(229,229,229)',
    xaxis=dict(
        gridcolor='rgb(255,255,255)',
        range=[1,150],
        showgrid=True,
        showline=False,
        showticklabels=True,
        tickcolor='rgb(127,127,127)',
        ticks='outside',
        zeroline=False,
        title = "epoch"
    ),
    yaxis=dict(
        gridcolor='rgb(255,255,255)',
        showgrid=True,
        showline=False,
        showticklabels=True,
        tickcolor='rgb(127,127,127)',
        ticks='outside',
        zeroline=False,
        range=[0, 1],
        title = "accuracy"
    ),
)

In [ ]:
fig = go.Figure(data=traces, layout=layout)
py.iplot(fig, filename='td_medium_lr_1_vanilla')

#### Validation accuraccy

In [ ]:
traces = []
metric = "val_acc"
for key,val in loss_plots.items():
    
    x = list(np.arange(1,len(val[0])+1))
    x_rev = x[::-1]
        
    metric_values = np.asarray([[epoch[metric] for epoch in run] for run in val])
    
    y_upper = list(np.amax(metric_values,axis=0))
    y_lower = np.amin(metric_values,axis=0)
    y_lower = list(y_lower[::-1])
    
    traces.append(go.Scatter(
        x=x+x_rev,
        y=y_upper+y_lower,
        fill='tozerox',
        #fillcolor='rgba(0,100,80,0.2)',
        #line=dict(color='rgba(255,255,255,0)'),
        showlegend=True,
        name=key,
    ))

In [ ]:
layout = go.Layout(
    paper_bgcolor='rgb(255,255,255)',
    plot_bgcolor='rgb(229,229,229)',
    xaxis=dict(
        gridcolor='rgb(255,255,255)',
        range=[1,150],
        showgrid=True,
        showline=False,
        showticklabels=True,
        tickcolor='rgb(127,127,127)',
        ticks='outside',
        zeroline=False,
        title = "epoch"
    ),
    yaxis=dict(
        gridcolor='rgb(255,255,255)',
        showgrid=True,
        showline=False,
        showticklabels=True,
        tickcolor='rgb(127,127,127)',
        ticks='outside',
        zeroline=False,
        range=[0, 1],
        title = "accuracy"
    ),
)

In [ ]:
fig = go.Figure(data=traces, layout=layout)
py.iplot(fig, filename='td_medium_lr_1_vanilla_validation')

### Accuracy experiment 2

In [ ]:
loss_plots = {}
for file in os.listdir("../logs/flowers_pretrained/"):
    opt = file.split("_")[2]
    
    if opt in loss_plots:
        loss_plots[opt].append(pickle.load(open(os.path.join("../logs","flowers_pretrained",file),"rb")))
    else:
        loss_plots[opt] = [pickle.load(open(os.path.join("../logs","flowers_pretrained",file),"rb"))]


#### Training accuracy

In [ ]:
traces = []
metric = "acc_train"
for key,val in loss_plots.items():
    
    x = list(np.arange(1,len(val[0])+1))
    x_rev = x[::-1]
        
    metric_values = np.asarray([[epoch[metric] for epoch in run] for run in val])
    
    y_upper = list(np.amax(metric_values,axis=0))
    y_lower = np.amin(metric_values,axis=0)
    y_lower = list(y_lower[::-1])
    
    traces.append(go.Scatter(
        x=x+x_rev,
        y=y_upper+y_lower,
        fill='tozerox',
        #fillcolor='rgba(0,100,80,0.2)',
        #line=dict(color='rgba(255,255,255,0)'),
        showlegend=True,
        name=key,
        
    ))

In [ ]:
layout = go.Layout(
    paper_bgcolor='rgb(255,255,255)',
    plot_bgcolor='rgb(229,229,229)',
    xaxis=dict(
        gridcolor='rgb(255,255,255)',
        range=[1,100],
        showgrid=True,
        showline=False,
        showticklabels=True,
        tickcolor='rgb(127,127,127)',
        ticks='outside',
        zeroline=False,
        title = "epoch"
    ),
    yaxis=dict(
        gridcolor='rgb(255,255,255)',
        showgrid=True,
        showline=False,
        showticklabels=True,
        tickcolor='rgb(127,127,127)',
        ticks='outside',
        zeroline=False,
        range=[0, 1],
        title = "accuracy"
    ),
)

In [ ]:
fig = go.Figure(data=traces, layout=layout)
py.iplot(fig, filename='td_medium_lr_1_pretrained')

#### Validation accuracy

In [ ]:
traces = []
metric = "acc_val"
for key,val in loss_plots.items():
    
    x = list(np.arange(1,len(val[0])+1))
    x_rev = x[::-1]
        
    metric_values = np.asarray([[epoch[metric] for epoch in run] for run in val])
    
    y_upper = list(np.amax(metric_values,axis=0))
    y_lower = np.amin(metric_values,axis=0)
    y_lower = list(y_lower[::-1])
    
    traces.append(go.Scatter(
        x=x+x_rev,
        y=y_upper+y_lower,
        fill='tozerox',
        #fillcolor='rgba(0,100,80,0.2)',
        #line=dict(color='rgba(255,255,255,0)'),
        showlegend=True,
        name=key,
        
    ))

In [ ]:
layout = go.Layout(
    paper_bgcolor='rgb(255,255,255)',
    plot_bgcolor='rgb(229,229,229)',
    xaxis=dict(
        gridcolor='rgb(255,255,255)',
        range=[1,100],
        showgrid=True,
        showline=False,
        showticklabels=True,
        tickcolor='rgb(127,127,127)',
        ticks='outside',
        zeroline=False,
        title = "epoch"
    ),
    yaxis=dict(
        gridcolor='rgb(255,255,255)',
        showgrid=True,
        showline=False,
        showticklabels=True,
        tickcolor='rgb(127,127,127)',
        ticks='outside',
        zeroline=False,
        range=[0, 1],
        title = "accuracy"
    ),
)

In [ ]:
fig = go.Figure(data=traces, layout=layout)
py.iplot(fig, filename='td_medium_lr_1_pretrained_validation')